# Geo analytics AAA - Intentionally Blank

Census_tracts_borders = 
https://www.census.gov/cgi-bin/geo/shapefiles/index.php?year=2016&layergroup=Census+Tracts

In [ ]:
import json

import pandas as pd
import numpy as np

import folium
import h3
import matplotlib.pyplot as plt
import geopandas as gpd
import branca.colormap as cm
import shapely
from shapely.geometry import shape

In [ ]:
taxi_df = pd.read_parquet('data/prepared/taxi_data_prepared.gzip')
taxi_df['pickup_centroid_location'] = gpd.GeoSeries.from_wkt(taxi_df['pickup_centroid_location'])
taxi_df['dropoff_centroid_location'] = gpd.GeoSeries.from_wkt(taxi_df['dropoff_centroid_location'])
census_tract_borders = gpd.read_file('data/chicago_census_tract_borders.zip')

In [ ]:
poi_df = gpd.read_file('data/POI.geojson')

In [ ]:
poi_df = poi_df.drop(columns= poi_df.columns.difference(["amenity", "bar", "brewery", "building", "construction:railway", "ferry", "healthcare", "public_transport", "railway", "station", "geometry"]))

In [ ]:
poi_df

In [ ]:
taxi_df

In [ ]:
census_tract_borders = census_tract_borders.drop(census_tract_borders.columns.difference(['GEOID', 'geometry']), axis=1)

In [ ]:
unique_census_tract_id = np.append(taxi_df['pickup_census_tract'].unique(), taxi_df['dropoff_census_tract'].unique()).astype('str')
census_tract_borders= census_tract_borders[census_tract_borders['GEOID'].isin(unique_census_tract_id)].reset_index(drop=True)
census_tract_borders

In [ ]:
def plotByFeatureStatic(dataframe, location='pickup', feature='all', aggregation='sum', missingCensusTract=False):
    """ Plot a feature of a dataframe on a map.

    Parameters
    ----------

    dataframe :  (pandas.DataFrame) 
        The dataframe to plot.
    location : (str) 
        The location column of the dataframe. Can be either 'pickup' or 'dropoff'. Default is 'pickup'.
    feature : (str) 
        The feature to aggregate. If 'all', all features are aggregated. Default is 'all'
    aggregation : (str)  
        The aggregation function to use. Can be either 'mean', 'median', 'sum', 'count', 'min', 'max'. Default is 'sum'.
    containMissingCensusTract : (bool)
        If True, census tracts with no data are included in the plot. Default is False.

    Returns
    ----------

    dataframe_grouped : (geopandas.GeoDataFrame) 
        The geodataframe grouped by the location column and the feature column. Contains always a geometry column and trip_count column.
    """
    dataframe_grouped = dataframe.copy()
    dataframe_grouped.dropna(inplace = True)

    if feature == 'all':
        features = dataframe_grouped.columns.difference(['pickup_census_tract', 'dropoff_census_tract', 'pickup_centroid_location', 'dropoff_centroid_location', 'trip_start_timestamp', 'trip_end_timestamp', 'taxi_id']).tolist()
    else:
        features = [feature]
    
    if location == 'pickup':
        features.append('pickup_census_tract')
        features.append('pickup_centroid_location')
        dataframe_grouped = dataframe_grouped.drop(columns=dataframe_grouped.columns.difference(features))
    elif location == 'dropoff':
        features.append('dropoff_census_tract')
        features.append('dropoff_centroid_location')
        dataframe_grouped = dataframe_grouped.drop(columns=dataframe_grouped.columns.difference(features))
    else:
        raise ValueError("Location must be either 'pickup' or 'dropoff'.")
    
    dataframe_grouped['trip_count'] = dataframe_grouped[features[0]]
    
    dataframe_grouped = dataframe_grouped.groupby([location + '_census_tract', location + '_centroid_location']).agg(lambda column: column.agg('count') if column.name == 'trip_count' else column.agg(aggregation)).reset_index()
    dataframe_grouped = dataframe_grouped.rename(columns={location + '_census_tract': 'GEOID'})
    dataframe_grouped['GEOID'] = dataframe_grouped['GEOID'].astype('str')

    if missingCensusTract == True:
        dataframe_grouped = dataframe_grouped.merge(census_tract_borders, on='GEOID', how='right')
        dataframe_grouped = gpd.GeoDataFrame(dataframe_grouped)
        dataframe_grouped[location + '_centroid_location'] = dataframe_grouped['geometry'].to_crs('+proj=cea').centroid.to_crs('EPSG:4326')
        dataframe_grouped['geometry'] = dataframe_grouped['geometry'].to_crs('EPSG:4326')
    else:
        dataframe_grouped = dataframe_grouped.merge(census_tract_borders, on='GEOID', how='left')
        dataframe_grouped = gpd.GeoDataFrame(dataframe_grouped)
        dataframe_grouped[location + '_centroid_location'] = dataframe_grouped['geometry'].to_crs('+proj=cea').centroid.to_crs('EPSG:4326')
        dataframe_grouped['geometry'] = dataframe_grouped['geometry'].to_crs('EPSG:4326')

    dataframe_grouped['poi_count'] = dataframe_grouped.apply(lambda row: row['geometry'].contains(poi_df['geometry']).sum(), axis=1)    
    return dataframe_grouped


In [ ]:
geo_df_taxi_pickup_sum = plotByFeatureStatic(taxi_df, missingCensusTract = True, feature='all', aggregation= "sum", location='pickup')

In [ ]:
geo_df_taxi_pickup_sum["poi_count"].sum()

In [ ]:
poi_in_census_tract = gpd.sjoin(poi_df, geo_df_taxi_pickup_sum, how='inner', op='within')

In [ ]:
loc = 'POI count by census tract pickup location logaritmic scale'
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(loc)   

m = folium.Map(location=(41.8,-87.723177), zoom_start=11)

m.get_root().html.add_child(folium.Element(title_html))
geo_df_taxi_pickup_sum[geo_df_taxi_pickup_sum['poi_count'] > 0].explore(column=np.log10(geo_df_taxi_pickup_sum[geo_df_taxi_pickup_sum['poi_count'] > 0]['poi_count']), tooltip=True, cmap="viridis", m = m, style_kwds = {"opacity": 0.2})
poi_in_census_tract.explore(tooltip=True, cmap="viridis", m = m, style_kwds = {"opacity": 0.3, "red": "red", "fillOpacity": 0.3, "fillColor": 'red'}, marker_kwds = {"radius": 3})

In [ ]:
geo_df_taxi_dropoff_sum = plotByFeatureStatic(taxi_df, missingCensusTract = True, feature='all', aggregation= "sum", location='dropoff')

In [ ]:
loc = 'Trip count by census tract pickup location logaritmic scale'
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(loc)   

m = folium.Map(location=(41.8,-87.723177), zoom_start=11)

m.get_root().html.add_child(folium.Element(title_html))
geo_df_taxi_pickup_sum.explore(column=np.log10(geo_df_taxi_pickup_sum['trip_count']), tooltip=True, cmap="viridis", m = m)

In [ ]:
loc = 'Trip count by census tract dropoff location logaritmic scale'
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(loc)   

m = folium.Map(location=(41.8,-87.723177), zoom_start=11)

m.get_root().html.add_child(folium.Element(title_html))
geo_df_taxi_dropoff_sum.explore(column=np.log10(geo_df_taxi_dropoff_sum['trip_count']), tooltip=True, cmap="viridis", m = m)

In [ ]:
# Placeholder
loc = 'Trip total by census tract pickup location logarithmic scale'
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(loc)   

m = folium.Map(location=(41.8,-87.723177), zoom_start=11)

m.get_root().html.add_child(folium.Element(title_html))
geo_df_taxi_pickup_sum.explore(column=np.log10(geo_df_taxi_pickup_sum['trip_total']), tooltip=True, cmap="viridis", m = m)

In [ ]:
fig, axd = plt.subplot_mosaic([['trip_total', 'trip_seconds'],
                               ['trip_miles', 'idle_seconds'],
                               ['trip_count', 'trip_count']],
                              figsize=(18,30), layout="constrained")
fig.suptitle('Features plotted by census tract pickup location on a logarithmic scale', fontsize=16)
geo_df_taxi_pickup_sum.plot(column=np.log10(geo_df_taxi_pickup_sum['trip_total']), cmap="viridis", legend=True,missing_kwds={'color': 'lightgrey'}, ax=axd['trip_total'], legend_kwds={"label": "Sum of Revenue (in dollar)", "orientation": "horizontal"})
geo_df_taxi_pickup_sum.plot(column=np.log10(geo_df_taxi_pickup_sum['trip_seconds']), cmap="viridis", legend=True,missing_kwds={'color': 'lightgrey'}, ax=axd['trip_seconds'], legend_kwds={"label": "Sum of trip time in seconds", "orientation": "horizontal"})
geo_df_taxi_pickup_sum.plot(column=np.log10(geo_df_taxi_pickup_sum['trip_miles']), cmap="viridis", legend=True,missing_kwds={'color': 'lightgrey'}, ax=axd['trip_miles'], legend_kwds={"label": "Sum of trip miles in seconds", "orientation": "horizontal"})
geo_df_taxi_pickup_sum[geo_df_taxi_pickup_sum['idle_seconds'] > 0].plot(column=np.log10(geo_df_taxi_pickup_sum[geo_df_taxi_pickup_sum['idle_seconds'] > 0]['idle_seconds']),  cmap="viridis", legend=True,missing_kwds={'color': 'lightgrey'}, ax=axd['idle_seconds'], legend_kwds={"label": "Sum of idle time in seconds", "orientation": "horizontal"})
geo_df_taxi_pickup_sum.plot(column=np.log10(geo_df_taxi_pickup_sum['trip_count']), cmap="viridis", legend=True,missing_kwds={'color': 'lightgrey'}, ax=axd['trip_count'], legend_kwds={"label": "Sum of Trips", "orientation": "horizontal"})

In [ ]:
geo_df_taxi_pickup_mean = plotByFeatureStatic(taxi_df, missingCensusTract = True, feature='all', aggregation= "mean", location='pickup')

In [ ]:
fig, axd = plt.subplot_mosaic([['trip_total', 'trip_seconds'],
                               ['trip_miles', 'idle_seconds'],
                               ['trip_count', 'trip_count']],
                              figsize=(18,30), layout="constrained")
fig.suptitle('Features plotted by census tract pickup location', fontsize=16)
geo_df_taxi_pickup_mean.plot(column=geo_df_taxi_pickup_mean['trip_total'], cmap="viridis", legend=True,missing_kwds={'color': 'lightgrey'},vmax= geo_df_taxi_pickup_mean['trip_total'].quantile(0.75), ax=axd['trip_total'], legend_kwds={"label": "Sum of Revenue (in dollar)", "orientation": "horizontal"})
geo_df_taxi_pickup_mean.plot(column=geo_df_taxi_pickup_mean['trip_seconds'], cmap="viridis", legend=True,missing_kwds={'color': 'lightgrey'},vmax= geo_df_taxi_pickup_mean['trip_seconds'].quantile(0.75), ax=axd['trip_seconds'], legend_kwds={"label": "Sum of trip time in seconds", "orientation": "horizontal"})
geo_df_taxi_pickup_mean.plot(column=geo_df_taxi_pickup_mean['trip_miles'], cmap="viridis", legend=True,missing_kwds={'color': 'lightgrey'},vmax= geo_df_taxi_pickup_mean['trip_miles'].quantile(0.75), ax=axd['trip_miles'], legend_kwds={"label": "Sum of trip miles in seconds", "orientation": "horizontal"})
geo_df_taxi_pickup_mean[(geo_df_taxi_pickup_mean['idle_seconds'] > 0) | geo_df_taxi_pickup_mean['idle_seconds'].isna()].plot(column=geo_df_taxi_pickup_mean[(geo_df_taxi_pickup_mean['idle_seconds'] > 0) | geo_df_taxi_pickup_mean['idle_seconds'].isna()]['idle_seconds'],  cmap="viridis", legend=True,missing_kwds={'color': 'lightgrey'},vmax= geo_df_taxi_pickup_mean['idle_seconds'].quantile(0.75), ax=axd['idle_seconds'], legend_kwds={"label": "Sum of idle time in seconds", "orientation": "horizontal"})
geo_df_taxi_pickup_sum.plot(column=np.log10(geo_df_taxi_pickup_sum['trip_count']), cmap="viridis", legend=True,missing_kwds={'color': 'lightgrey'}, ax=axd['trip_count'], legend_kwds={"label": "Sum of Trips", "orientation": "horizontal"})

In [ ]:
# Create docstring for the following function
def plotH3_HexagonMap(dataframe, location='pickup', feature='all', aggregation='sum', missingCensusTract=False, hexRes = 10):
    """ Plot a feature of a dataframe on a map.

    Parameters
    ----------

    dataframe :  (pandas.DataFrame) 
        The dataframe to plot.
    location : (str) 
        The location column of the dataframe. Can be either 'pickup' or 'dropoff'. Default is 'pickup'.
    feature : (str) 
        The feature to aggregate. If 'all', all features are aggregated. Default is 'all'
    aggregation : (str)  
        The aggregation function to use. Can be either 'mean', 'median', 'sum', 'count', 'min', 'max'. Default is 'sum'.
    containMissingCensusTract : (bool)
        If True, census tracts with no data are included in the plot. Default is False.
    hexRes : (int)
        H3 hexagon resolution size. Default is 10.
    Returns
    ----------

    taxi_df_geo_grouped : (geopandas.GeoDataFrame) 
        The geodataframe grouped by the location column and the feature column. Contains always a geometry column and trip_count column.
    """
    taxi_df_geo = plotByFeatureStatic(dataframe, location= location,feature = feature, aggregation = aggregation, missingCensusTract = missingCensusTract)
    # geometry to h3 index
    taxi_df_geo['h3_index'] = taxi_df_geo.apply(lambda row: h3.geo_to_h3(row[location + '_centroid_location'].y, row[location + '_centroid_location'].x, hexRes), axis=1)

    geojson = []
    geometries = []
    indexes = []

    for geometry in taxi_df_geo['geometry']:
        geojson.append(shapely.to_geojson(geometry))

    for geometry in geojson:
        obj = json.loads(geometry)
        h3_indexes = h3.polyfill(obj, hexRes ,True)
        for index in h3_indexes:
            geometries.append(shape({"type": "Polygon",
                    "coordinates": [h3.h3_to_geo_boundary(index, geo_json=True)],
                    "properties": ""
                    }))
            indexes.append(index)   
    taxi_df_geo.drop(columns= ['geometry', 'pickup_centroid_location', 'GEOID'], inplace = True)
    df_h3_polyfilled = pd.DataFrame({'h3_index': indexes})
    taxi_df_geo_grouped = taxi_df_geo.groupby('h3_index').agg(aggregation).reset_index()
    taxi_df_geo_grouped = taxi_df_geo_grouped.merge(df_h3_polyfilled, on='h3_index', how='outer')
    taxi_df_geo_grouped['geometry'] = taxi_df_geo_grouped.apply(lambda row: shape({"type": "Polygon",
                                           "coordinates": [h3.h3_to_geo_boundary(row["h3_index"], geo_json=True)],
                                           "properties": ""
                                           }), axis=1)
    taxi_df_geo_grouped.loc[taxi_df_geo_grouped['trip_count'] == 0,'trip_count'] = np.nan
    taxi_df_geo_grouped.loc[taxi_df_geo_grouped['trip_miles'] == 0,'trip_miles'] = np.nan
    taxi_df_geo_grouped.loc[taxi_df_geo_grouped['trip_seconds'] == 0,'trip_seconds'] = np.nan
    taxi_df_geo_grouped.loc[taxi_df_geo_grouped['trip_total'] == 0,'trip_total'] = np.nan
    taxi_df_geo_grouped = gpd.GeoDataFrame(taxi_df_geo_grouped, crs='EPSG:4326', geometry='geometry')

    taxi_df_geo_grouped['poi_count'] = taxi_df_geo_grouped.apply(lambda row: row['geometry'].contains(poi_df['geometry']).sum(), axis=1)    
    return taxi_df_geo_grouped


In [ ]:
geo_df_taxi_h3_pickup_sum_res4 = plotH3_HexagonMap(taxi_df, location='pickup', feature='all', aggregation='sum', missingCensusTract=True, hexRes= 4)

In [ ]:
loc = 'Trip count by h3 hexagon pickup location logaritmic scale'
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(loc)   

m = folium.Map(location=(41.8,-87.723177), zoom_start=11)

m.get_root().html.add_child(folium.Element(title_html))
geo_df_taxi_h3_pickup_sum_res4.explore(column=geo_df_taxi_h3_pickup_sum_res4['trip_count'], vmax = geo_df_taxi_h3_pickup_sum_res4['trip_count'].quantile(0.75), tooltip=True, cmap="viridis", m = m)

In [ ]:
geo_df_taxi_h3_pickup_sum_res7 = plotH3_HexagonMap(taxi_df, location='pickup', feature='all', aggregation='sum', missingCensusTract=True, hexRes= 7)

In [ ]:
# Placeholder
loc = 'Trip total by h3 hexagon pickup location logarithmic scale'
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(loc)   

m = folium.Map(location=(41.8,-87.723177), zoom_start=11)

m.get_root().html.add_child(folium.Element(title_html))
geo_df_taxi_h3_pickup_sum_res7.explore(column=np.log10(geo_df_taxi_h3_pickup_sum_res7['trip_total']), tooltip=True, cmap="viridis", m = m)

In [ ]:
geo_df_taxi_h3_pickup_sum_res8 = plotH3_HexagonMap(taxi_df, location='pickup', feature='all', aggregation='sum', missingCensusTract=True, hexRes= 8)

In [ ]:
# Placeholder
loc = 'Trip total by h3 hexagon pickup location logarithmic scale'
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(loc)   

m = folium.Map(location=(41.8,-87.723177), zoom_start=11)

m.get_root().html.add_child(folium.Element(title_html))
geo_df_taxi_h3_pickup_sum_res8.explore(column=np.log10(geo_df_taxi_h3_pickup_sum_res8['trip_total']), tooltip=True, cmap="viridis", m = m)

In [ ]:
geo_df_taxi_h3_pickup_sum_res9 = plotH3_HexagonMap(taxi_df, location='pickup', feature='all', aggregation='sum', missingCensusTract=True, hexRes= 9)

In [ ]:
# Placeholder
loc = 'Trip total by h3 hexagon pickup location logarithmic scale'
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(loc)   

m = folium.Map(location=(41.8,-87.723177), zoom_start=11)

m.get_root().html.add_child(folium.Element(title_html))
geo_df_taxi_h3_pickup_sum_res9.explore(column=np.log10(geo_df_taxi_h3_pickup_sum_res9['trip_total']), tooltip=True, cmap="viridis", m = m)

In [ ]:
# Placeholder
loc = 'Trip total by census tract pickup location logarithmic scale'
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(loc)   

m = folium.Map(location=(41.8,-87.723177), zoom_start=11)

m.get_root().html.add_child(folium.Element(title_html))
geo_df_taxi_h3_pickup_sum_res8.explore(column=np.log10(geo_df_taxi_h3_pickup_sum_res8['trip_total']), tooltip=True, cmap="viridis", m = m)

In [ ]:
fig, axd = plt.subplot_mosaic([['trip_total', 'trip_seconds'],
                               ['trip_miles', 'idle_seconds'],
                               ['trip_count', 'trip_count']],
                              figsize=(18,30), layout="constrained")
fig.suptitle('Features plotted by census tract pickup location on a logarithmic scale', fontsize=16)
geo_df_taxi_h3_pickup_sum_res8.plot(column=np.log10(geo_df_taxi_h3_pickup_sum_res8['trip_total']), cmap="viridis", legend=True,missing_kwds={'color': 'lightgrey'}, ax=axd['trip_total'], legend_kwds={"label": "Sum of Revenue (in dollar)", "orientation": "horizontal"})
geo_df_taxi_h3_pickup_sum_res8.plot(column=np.log10(geo_df_taxi_h3_pickup_sum_res8['trip_seconds']), cmap="viridis", legend=True,missing_kwds={'color': 'lightgrey'}, ax=axd['trip_seconds'], legend_kwds={"label": "Sum of trip time in seconds", "orientation": "horizontal"})
geo_df_taxi_h3_pickup_sum_res8.plot(column=np.log10(geo_df_taxi_h3_pickup_sum_res8['trip_miles']), cmap="viridis", legend=True,missing_kwds={'color': 'lightgrey'}, ax=axd['trip_miles'], legend_kwds={"label": "Sum of trip miles in seconds", "orientation": "horizontal"})
geo_df_taxi_h3_pickup_sum_res8[(geo_df_taxi_h3_pickup_sum_res8['idle_seconds'] > 0) | geo_df_taxi_h3_pickup_sum_res8['idle_seconds'].isna()].plot(column=np.log10(geo_df_taxi_h3_pickup_sum_res8[(geo_df_taxi_h3_pickup_sum_res8['idle_seconds'] > 0) | geo_df_taxi_h3_pickup_sum_res8['idle_seconds'].isna()]['idle_seconds']),  cmap="viridis", legend=True,missing_kwds={'color': 'lightgrey'}, ax=axd['idle_seconds'], legend_kwds={"label": "Sum of idle time in seconds", "orientation": "horizontal"})
geo_df_taxi_h3_pickup_sum_res8.plot(column=np.log10(geo_df_taxi_h3_pickup_sum_res8['trip_count']), cmap="viridis", legend=True,missing_kwds={'color': 'lightgrey'}, ax=axd['trip_count'], legend_kwds={"label": "Sum of Trips", "orientation": "horizontal"})

In [ ]:
geo_df_taxi_h3_pickup_mean_res8 = plotH3_HexagonMap(taxi_df, location='pickup', feature='all', aggregation='mean', missingCensusTract=True, hexRes= 8)

In [ ]:
fig, axd = plt.subplot_mosaic([['trip_total', 'trip_seconds'],
                               ['trip_miles', 'idle_seconds'],
                               ['trip_count', 'trip_count']],
                              figsize=(18,30), layout="constrained")
fig.suptitle('Features plotted by census tract pickup location', fontsize=16)
geo_df_taxi_h3_pickup_mean_res8.plot(column=geo_df_taxi_h3_pickup_mean_res8['trip_total'], cmap="viridis", legend=True,missing_kwds={'color': 'lightgrey'},vmax= geo_df_taxi_h3_pickup_mean_res8['trip_total'].quantile(0.75), ax=axd['trip_total'], legend_kwds={"label": "Sum of Revenue (in dollar)", "orientation": "horizontal"})
geo_df_taxi_h3_pickup_mean_res8.plot(column=geo_df_taxi_h3_pickup_mean_res8['trip_seconds'], cmap="viridis", legend=True,missing_kwds={'color': 'lightgrey'},vmax= geo_df_taxi_h3_pickup_mean_res8['trip_seconds'].quantile(0.75), ax=axd['trip_seconds'], legend_kwds={"label": "Sum of trip time in seconds", "orientation": "horizontal"})
geo_df_taxi_h3_pickup_mean_res8.plot(column=geo_df_taxi_h3_pickup_mean_res8['trip_miles'], cmap="viridis", legend=True,missing_kwds={'color': 'lightgrey'},vmax= geo_df_taxi_h3_pickup_mean_res8['trip_miles'].quantile(0.75), ax=axd['trip_miles'], legend_kwds={"label": "Sum of trip miles in seconds", "orientation": "horizontal"})
geo_df_taxi_h3_pickup_mean_res8[(geo_df_taxi_h3_pickup_mean_res8['idle_seconds'] > 0) | geo_df_taxi_h3_pickup_mean_res8['idle_seconds'].isna()].plot(column=geo_df_taxi_h3_pickup_mean_res8[(geo_df_taxi_h3_pickup_mean_res8['idle_seconds'] > 0) | geo_df_taxi_h3_pickup_mean_res8['idle_seconds'].isna()]['idle_seconds'],  cmap="viridis", legend=True,missing_kwds={'color': 'lightgrey'},vmax= geo_df_taxi_pickup_mean['idle_seconds'].quantile(0.75), ax=axd['idle_seconds'], legend_kwds={"label": "Sum of idle time in seconds", "orientation": "horizontal"})
geo_df_taxi_h3_pickup_mean_res8.plot(column=np.log10(geo_df_taxi_h3_pickup_mean_res8['trip_count']), cmap="viridis", legend=True,missing_kwds={'color': 'lightgrey'}, ax=axd['trip_count'], legend_kwds={"label": "Sum of Trips", "orientation": "horizontal"})